In [1]:
import findspark
findspark.init()

from pyspark.ml.feature import Word2Vec
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext

from pyspark import SparkContext
from pyspark.streaming import StreamingContext #Import streaming context
from pyspark.sql import SparkSession

import spacy
from pyspark.sql import functions as F
from pyspark.ml.functions import array_to_vector

import numpy as np
import spacy

import pandas as pd
import numpy as np
import torch.nn.functional as F
import torch.nn as nn
import torch
import spacy
import math

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from tqdm.auto import tqdm
from torch.utils.data import DataLoader
from torch import nn

tqdm.pandas()
encoder = LabelEncoder()

nlp = spacy.load('en_core_web_lg')

2022-12-07 20:41:51.490975: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-07 20:41:53.353566: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-07 20:41:53.353634: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-12-07 20:41:53.353641: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
spark = SparkSession.builder\
    .master('local[*]')\
    .appName('explore')\
    .getOrCreate()
sc = spark.sparkContext._conf.setAll([('spark.driver.maxResultSize', '8g')])

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/12/07 20:42:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
if torch.cuda.is_available():
  print('Good to go!')
else:
  print('Please set GPU via Edit -> Notebook Settings.')
device = torch.device('cuda:0')

Good to go!


In [4]:
df = spark.read.json('/common/users/shared/cs543_fall22_group3/combined/combined_raw')

In [5]:
selected = pd.DataFrame(df.take(640))

In [6]:
selected.columns = ['text']

In [7]:
selected['vector'] = selected['text'].progress_apply(lambda x: nlp(x).vector)

100%|██████████| 640/640 [00:03<00:00, 189.13it/s]


In [10]:
# function to create batches of our data
def collate_batch(batch):
    embedding_list = []
    
    for (_embedding) in batch:
        embedding = torch.tensor(_embedding, dtype=torch.float32)
        embedding_list.append(embedding)

    embedding_list = torch.stack(embedding_list)
    return embedding_list.to(device)

In [11]:
class TextClassificationModel(nn.Module):

    def __init__(self, num_class, embed_dim=300, vocab_size=45, pad_index=0,
                 stride=1, kernel_size=3, conv_out_size=64, dropout_rate=0.25):
        super(TextClassificationModel, self).__init__()

        # Embedding layer parameters
        self.embed_size = embed_dim
        self.vocab_size = vocab_size
        self.pad_index = pad_index
       
        # Conv layer parameters
        self.stride = stride
        self.kernel_size = kernel_size
        self.conv_out_size = conv_out_size
       
        # Misc
        self.dropout_rate = dropout_rate
        
        self.embed_size = 1
        # Layers
        self.conv = torch.nn.Conv1d(self.embed_size, self.conv_out_size, self.kernel_size, self.stride)
        self.hidden_act = torch.relu
        self.max_pool = torch.nn.MaxPool1d(self.kernel_size, self.stride)
       
        self.flatten = lambda x: x.view(x.shape[0], x.shape[1]*x.shape[2])
       
        self.fc = torch.nn.Linear(self._linear_layer_in_size(), num_class)

        if self.dropout_rate:
            self.dropout = torch.nn.Dropout(self.dropout_rate)

    def _linear_layer_in_size(self):
        out_conv_1 = ((self.embed_size - 1 * (self.kernel_size - 1) - 1) / self.stride) + 1
        out_conv_1 = math.floor(out_conv_1)
        out_pool_1 = ((out_conv_1 - 1 * (self.kernel_size - 1) - 1) / self.stride) + 1
        out_pool_1 = math.floor(out_pool_1)
                           
        # return out_pool_1*self.conv_out_size
        return 18944

    def forward(self, x):
        # print(x.shape)

        # x = torch.reshape(x. (x.shape[0],)

        x = torch.unsqueeze(x, 1)
        # x = torch.transpose(x, 1, 2) # (batch, 1, 300)

        x = self.conv(x)
        # print(x.shape)

        x = self.hidden_act(x)
        # print(x.shape)

        x = self.max_pool(x)
        # print(x.shape)

        x = self.flatten(x)
        # print(x.shape)

        if self.dropout_rate:
            x = self.dropout(x)

        x = self.fc(x)

        return x

In [18]:
import torch
import math
m = TextClassificationModel(42)
m.load_state_dict(torch.load('/common/users/shared/cs543_fall22_group3/models/class_model.pt'))
m.to(torch.device(device))

TextClassificationModel(
  (conv): Conv1d(1, 64, kernel_size=(3,), stride=(1,))
  (max_pool): MaxPool1d(kernel_size=3, stride=1, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=18944, out_features=42, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)

In [15]:
def predict(dataloader):
    m.eval()
    pred = []

    with torch.no_grad():
        for idx, vector in enumerate(dataloader):
            predicted_label = m(vector)
            pred = pred + list(predicted_label.argmax(1).cpu().detach().numpy())

    return pred

In [16]:
from torchtext.data.functional import to_map_style_dataset

classify_dataset = to_map_style_dataset(selected['vector'])

test_dataloader = DataLoader(classify_dataset, batch_size=64, shuffle=True, collate_fn=collate_batch)

In [19]:
pred = predict(test_dataloader)

In [22]:
selected['predictions'] = pred

In [24]:
category_dict = {
0: 'ARTS',
1: 'ARTS & CULTURE',
2: 'BLACK VOICES',
3: 'BUSINESS',
4: 'COLLEGE',
5: 'COMEDY',
6: 'CRIME',
7: 'CULTURE & ARTS',
8: 'DIVORCE',
9: 'EDUCATION',
10: 'ENTERTAINMENT',
11: 'ENVIRONMENT',
12: 'FIFTY',
13: 'FOOD & DRINK',
14: 'GOOD NEWS',
15: 'GREEN',
16: 'HEALTHY LIVING',
17: 'HOME & LIVING',
18: 'IMPACT',
19: 'LATINO VOICES',
20: 'MEDIA',
21: 'MONEY',
22: 'PARENTING',
23: 'PARENTS',
24: 'POLITICS',
25: 'QUEER VOICES',
26: 'RELIGION',
27: 'SCIENCE',
28: 'SPORTS',
29: 'STYLE',
30: 'STYLE & BEAUTY',
31: 'TASTE',
32: 'TECH',
33: 'THE WORLDPOST',
34: 'TRAVEL',
35: 'U.S. NEWS',
36: 'WEDDINGS',
37: 'WEIRD NEWS',
38: 'WELLNESS',
39: 'WOMEN',
40: 'WORLD NEWS',
41: 'WORLDPOST'}

In [26]:
selected['prediction_label'] = selected['predictions'].apply(lambda x: category_dict[x])

In [27]:
selected.head()

,text,vector,predictions,prediction_label
0,University of Hyderabad start-up develops stem...,"[-2.339089, -0.285064, -0.20233802, 3.4956944,...",34,TRAVEL
1,K Chandrasekhar Rao not for poor: Andhra Prade...,"[-1.5490905, 1.7642515, 1.1541762, -0.8794538,...",40,WORLD NEWS
2,Temple town Palitana gets 100-bed isolation fa...,"[-0.99294555, 0.19115537, -1.6017834, 1.909956...",17,HOME & LIVING
3,UDF candidate C Raghunath asks KPCC chief to quit,"[-0.17553778, 2.8221998, 0.65723646, -0.678131...",24,POLITICS
4,Greater Hyderabad Municipal Corporation begins...,"[-0.97658616, -1.1106362, 0.4652062, 0.1107862...",24,POLITICS
